МФТИ ФИВТ, Машинное обучение, Никита Волков

# Ранжирование

* Дедлайн **14 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями.
* Кагл: https://inclass.kaggle.com/c/ml-mipt-ranking4
* Инвайт: https://kaggle.com/join/4uthnui9sndi

### Небольшая предыстория

Внезапно очутившись на 4 курсе (как это произошло, вообще не понятно) студенты ФИВТа поняли, что времени стало совсем не хватать. Немного подумав, они решили, что во всем виноваты обезумевшие преподаватели курса Машинное обучение. Было проведено коллективное собрание, на котором было принято решение — попробовать договориться с преподавателями. Однако, все попытки примирения были отвергнуты. И тогда на очередном собрании (вместо выполнения очередного ДЗ ;)) было принято решение сделать вопросно-ответную систему для студентов. Например, она сможет четко отвечать на вопросы следующего типа ”Когда дедлайн по очередному огромному заданию?”, ”Что нужно сделать в задании?”, а вопрос ”Сколько суток нужно потратить, чтобы выполнить очередное ДЗ” даже никто и задавать не будет — никто их не делает, кроме самих преподавателей. А на вопрос ”Сможем ли мы переубедить Арсения Ашуху?” система ответит картинкой:

<img width=400 src="./pic.png">

Для каждого вопроса студенты смогли написать систему, которая подбирает наиболее подходящие ответы в интернете. А вот, как из упорядочить, они не знают, поскольку прогуляли лекцию по ранжированию. К счастью, было записано видео этой лекции, так что вы сможете им помочь.

А теперь серьезно...

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

### Бейзлайн

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np

Считываем все запросы из test

In [ ]:
with open('queries_test.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

По очереди загружаем все документы, считая для каждого TF и сортируя для каждого запроса. Это и есть скор в бейзлайне.

In [ ]:
with open('test.txt') as ft:
    with open('baseline-1.txt', 'w') as fb:
        ft.readline()
        fb.write('QueryId,DocumentId\n')

        qids, docids, relevs = [], [], []
        for line in ft:
            qid, docid, url = line.split(',')
            qid, docid = int(qid), int(docid)
            
            if len(qids) > 0 and qid != qids[-1]:
                order = np.argsort(relevs)[::-1]
                qids = np.array(qids)[order]
                docids = np.array(docids)[order]
                
                for i in range(len(qids)):
                    fb.write('{},{}\n'.format(qids[i], docids[i]))
                    
                qids, docids, relevs = [], [], []

            text = ' '.join(BeautifulSoup(urlopen(url, timeout=10), 'lxml').stripped_strings)
            text = np.array(text.split(' '))

            frec = 0
            for word in queries[qid - 101].split(' '):
                frec += (text == word).mean()
                
            qids.append(qid)
            docids.append(docid)
            relevs.append(frec)

Если у вас не загружаются некоторые страницы, подумайте, как с этим можно бороться. Например, посмотрите код с семинара и из предыдущего ДЗ. Если и это не помогает, то нужно подумать, что делать в этой ситуации.

### Кажется, это последнее задание

Свобода!

<img width=200 src="./last.jpg">

А, нет, еще экзамен...